Kobert.ipynb 와 DCGAN.ipynb에서 훈련한 모델들을 불러온다.  
가중치만 저장했기 때문에 Model architecture는 선언해주어야한다.  
따라서 필요한 라이브러리와 Pretrained model을 import한다.

# KOBERT

In [ ]:
!pip install 'git+https://github.com/SKTBrain/KoBERT.git#egg=kobert_tokenizer&subdirectory=kobert_hf'
!pip install gluonnlp pandas tqdm
!pip install mxnet
!pip install sentencepiece

In [ ]:
import torch
from torch import nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
import numpy as np
import pandas as pd
import gluonnlp as nlp
from tqdm import tqdm, tqdm_notebook
from transformers import AdamW
from transformers.optimization import get_cosine_schedule_with_warmup
from transformers import BertModel
from kobert_tokenizer import KoBERTTokenizer
from torchvision import datasets
from torchvision.transforms.functional import to_pil_image
import torchvision.transforms as transforms
import torch.nn.functional as F
import os
import time
import matplotlib.pyplot as plt
%matplotlib inline
from __future__ import print_function
#%matplotlib inline
import argparse
import random
import torch.nn as nn
import torch.nn.parallel
import torch.backends.cudnn as cudnn
import torch.utils.data
import torchvision.datasets as dset
import torchvision.utils as vutils
import matplotlib.animation as animation
from IPython.display import HTML
from PIL import Image

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu') #GPU 사용

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
#Dataset을 정의하는 Class
class BERTDataset(Dataset):
    def __init__(self, dataset, sent_idx, label_idx, bert_tokenizer, vocab, max_len,
                 pad, pair):
        transform = nlp.data.BERTSentenceTransform(
            bert_tokenizer, max_seq_length=max_len, vocab=vocab, pad=pad, pair=pair)

        self.sentences = [transform([i[sent_idx]]) for i in dataset]
        self.labels = [np.int32(i[label_idx]) for i in dataset]

    def __getitem__(self, i):
        return (self.sentences[i] + (self.labels[i], ))

    def __len__(self):
        return (len(self.labels))

In [ ]:
tokenizer = KoBERTTokenizer.from_pretrained('skt/kobert-base-v1')
bertmodel = BertModel.from_pretrained('skt/kobert-base-v1', return_dict=False)
vocab = nlp.vocab.BERTVocab.from_sentencepiece(tokenizer.vocab_file, padding_token='[PAD]')
tok = nlp.data.BERTSPTokenizer(tokenizer, vocab, lower = False)

In [ ]:
tok = tokenizer.tokenize

KOBERT ARCHITECTURE

In [ ]:
class BERTClassifier(nn.Module):
    def __init__(self,
                 bert,
                 hidden_size = 768,
                 num_classes = 4,
                 dr_rate=None,
                 params=None):
        super(BERTClassifier, self).__init__()
        self.bert = bert
        self.dr_rate = dr_rate

        self.classifier = nn.Linear(hidden_size , num_classes)
        if dr_rate:
            self.dropout = nn.Dropout(p=dr_rate)

    def gen_attention_mask(self, token_ids, valid_length):
        attention_mask = torch.zeros_like(token_ids)
        for i, v in enumerate(valid_length):
            attention_mask[i][:v] = 1
        return attention_mask.float()

    def forward(self, token_ids, valid_length, segment_ids):
        attention_mask = self.gen_attention_mask(token_ids, valid_length)

        _, pooler = self.bert(input_ids = token_ids, token_type_ids = segment_ids.long(), attention_mask = attention_mask.float().to(token_ids.device))
        if self.dr_rate:
            out = self.dropout(pooler)
        return self.classifier(out)

In [ ]:
#Hyper parameter
max_len = 64
batch_size = 64
warmup_ratio = 0.1
num_epochs = 10
max_grad_norm = 1
log_interval = 200
learning_rate =  5e-5

In [ ]:
#훈련한 모델 weight 불러오기
filepath = '/content/drive/MyDrive/aipro/new_bert.pth'
model_bert = BERTClassifier(bertmodel,dr_rate = 0.5)
model_bert.load_state_dict(torch.load(filepath,map_location = torch.device('cpu')))

In [ ]:
def new_softmax(a) :
    c = np.max(a) # 최댓값
    exp_a = np.exp(a-c) # 각각의 원소에 최댓값을 뺀 값에 exp를 취한다. (이를 통해 overflow 방지)
    sum_exp_a = np.sum(exp_a)
    y = (exp_a / sum_exp_a) * 100
    return np.round(y, 3)

def predict(predict_sentence): # 모델을 예측할 때, 입력으로 그냥 문장을 넣으면 안됨, Token화 된 문장을 넣어야하므로 Predict함수를 따로 정의하여 처리해주어야한다

    data = [predict_sentence, '0']
    dataset_another = [data]

    another_test = BERTDataset(dataset_another, 0, 1, tok, vocab, max_len, True, False)
    test_dataloader = torch.utils.data.DataLoader(another_test, batch_size=batch_size, num_workers=5)

    model_bert.eval()

    for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(test_dataloader):
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)

        valid_length= valid_length
        label = label.long().to(device)

        out = model_bert(token_ids, valid_length, segment_ids)


        test_eval=[]
        for i in out:
            logits=i
            logits = logits.detach().cpu().numpy()
            min_v = min(logits)
            total = 0
            probability = []
            logits = np.round(new_softmax(logits),3).tolist()
            for logit in logits:
                probability.append(np.round(logit,3))

            if np.argmax(logits) == 0:
                test_eval.append("분노")
            elif np.argmax(logits) == 1:
                test_eval.append("슬픔")
            elif np.argmax(logits) == 2:
                test_eval.append("불안")
            elif np.argmax(logits) == 3:
                test_eval.append("행복")
            #print(probability)

        return test_eval[0]

# DCGAN

In [ ]:
# transforms 정의하기
h, w = 64, 64
mean = (0.5, 0.5, 0.5)
std = (0.5, 0.5, 0.5)

transform = transforms.Compose([
                    transforms.Resize((h,w)),
                    transforms.ToTensor(),
                    transforms.Normalize(mean, std)
])

In [ ]:
# 파라미터 정의
params = {'nz':100, # noise 수
          'ngf':64, # generator에서 사용하는 conv filter 수
          'ndf':64, # discriminator에서 사용하는 conv filter 수
          'img_channel':3, # 이미지 채널
          }

In [ ]:
# Generator: noise를 입력받아 가짜 이미지를 생성합니다.
class Generator(nn.Module):
    def __init__(self, params):
        super().__init__()
        nz = params['nz'] # noise 수, 100
        ngf = params['ngf'] # conv filter 수
        img_channel = params['img_channel'] # 이미지 채널

        self.dconv1 = nn.ConvTranspose2d(nz,ngf*8,4, stride=1, padding=0, bias=False)
        self.bn1 = nn.BatchNorm2d(ngf*8)
        self.dconv2 = nn.ConvTranspose2d(ngf*8,ngf*4, 4, stride=2, padding=1, bias=False)
        self.bn2 = nn.BatchNorm2d(ngf*4)
        self.dconv3 = nn.ConvTranspose2d(ngf*4,ngf*2,4,stride=2,padding=1,bias=False)
        self.bn3 = nn.BatchNorm2d(ngf*2)
        self.dconv4 = nn.ConvTranspose2d(ngf*2,ngf,4,stride=2,padding=1,bias=False)
        self.bn4 = nn.BatchNorm2d(ngf)
        self.dconv5 = nn.ConvTranspose2d(ngf,img_channel,4,stride=2,padding=1,bias=False)

    def forward(self,x):
        x = F.relu(self.bn1(self.dconv1(x)))
        x = F.relu(self.bn2(self.dconv2(x)))
        x = F.relu(self.bn3(self.dconv3(x)))
        x = F.relu(self.bn4(self.dconv4(x)))
        x = torch.tanh(self.dconv5(x))
        return x

# check
x = torch.randn(1,100,1,1, device=device)
model_gen = Generator(params).to(device)
out_gen = model_gen(x)
print(out_gen.shape)

In [ ]:
# Discriminator: 진짜 이미지와 가짜 이미지를 식별합니다.
class Discriminator(nn.Module):
    def __init__(self,params):
        super().__init__()
        img_channel = params['img_channel'] # 3
        ndf = params['ndf'] # 64

        self.conv1 = nn.Conv2d(img_channel,ndf,4,stride=2,padding=1,bias=False)
        self.conv2 = nn.Conv2d(ndf,ndf*2,4,stride=2,padding=1,bias=False)
        self.bn2 = nn.BatchNorm2d(ndf*2)
        self.conv3 = nn.Conv2d(ndf*2,ndf*4,4,stride=2,padding=1,bias=False)
        self.bn3 = nn.BatchNorm2d(ndf*4)
        self.conv4 = nn.Conv2d(ndf*4,ndf*8,4,stride=2,padding=1,bias=False)
        self.bn4 = nn.BatchNorm2d(ndf*8)
        self.conv5 = nn.Conv2d(ndf*8,1,4,stride=1,padding=0,bias=False)

    def forward(self,x):
        x = F.leaky_relu(self.conv1(x),0.2)
        x = F.leaky_relu(self.bn2(self.conv2(x)),0.2)
        x = F.leaky_relu(self.bn3(self.conv3(x)),0.2)
        x = F.leaky_relu(self.bn4(self.conv4(x)),0.2)
        x = torch.sigmoid(self.conv5(x))
        return x.view(-1,1)

# check
x = torch.randn(16,3,64,64,device=device)
model_dis = Discriminator(params).to(device)
out_dis = model_dis(x)
print(out_dis.shape)

In [ ]:
# 가중치 불러오기
path2models = '/content/drive/MyDrive/aipro'
path2weights_gen = os.path.join(path2models, 'weights_gen.pt')
path2weights_dis = os.path.join(path2models, 'weights_dis.pt')

weights = torch.load(path2weights_gen, map_location = torch.device('cpu'))
model_gen.load_state_dict(weights)

In [ ]:
# evalutaion mode
model_gen.eval()

In [ ]:
# fake image 생성 test
with torch.no_grad():
    fixed_noise = torch.randn(1, 100,1,1, device=device)
    label = torch.randint(0,10,(16,), device=device)
    img_fake = model_gen(fixed_noise).detach().cpu()

plt.imshow(to_pil_image(0.5*img_fake[0] + 0.5), cmap = 'gray')
plt.axis('off')

# 시연

In [ ]:
from googleapiclient.discovery import build

comfort_df = pd.read_csv('/content/drive/MyDrive/aipro/글귀.csv', encoding='cp949')
comfort_df = comfort_df.dropna()

def get_random_sentence(emotion):
    return comfort_df[emotion].sample().iloc[0]

def search_youtube(api_key, query, max_results=5):
    youtube = build('youtube', 'v3', developerKey=api_key)

    # 검색 실행
    search_response = youtube.search().list(
        q=query,
        type='video',
        part='id,snippet',
        maxResults=max_results
    ).execute()

    # 결과에서 동영상의 정보 추출
    video_info = [{'title': item['snippet']['title'], 'videoId': item['id']['videoId']} for item in search_response['items']]

    # 각 동영상의 주소와 제목 생성
    results = [{'title': info['title'], 'url': f'https://www.youtube.com/watch?v={info["videoId"]}'}
               for info in video_info]

    return results

if __name__ == "__main__":
    # YouTube API 키를 설정
    youtube_api_key = ""

    print("안녕하세요! 저는 당신만의 감정분석AI 티미에요 ( ˃ ⩌˂)\n")
    user_input = input("오늘 하루 기분은 어떠셨는지, 저에게 자유롭게 얘기해주세요!\n")
    predicted_emotion = predict(user_input)

    # 검색어를 입력
    if(predicted_emotion == "분노"):
        search_query = "화날 때 듣기 좋은 음악"
        comfort_sentence = get_random_sentence("분노")
    elif(predicted_emotion == "슬픔"):
        search_query = "슬플 때 듣기 좋은 음악"
        comfort_sentence = get_random_sentence("슬픔")
    elif(predicted_emotion == "불안"):
        search_query =  "불안할 때 듣기 좋은 음악"
        comfort_sentence = get_random_sentence("불안")
    elif(predicted_emotion == "행복"):
        search_query = "기분 좋을 때 듣기 좋은 음악"
        comfort_sentence = "행복"

    # YouTube API를 사용하여 동영상 주소 및 제목 가져오기
    video_results = search_youtube(youtube_api_key, search_query)
    # 결과 출력
    if(comfort_sentence):
        print("\n현재 감정 상태가","<",predicted_emotion,">","이시네요.")
        print("\n",predicted_emotion,"감정을 가진 당신을 위해 멋진 풍경 그림을 그려보았어요!")
        with torch.no_grad():
            fixed_noise = torch.randn(1, 100,1,1, device=device)
            label = torch.randint(0,10,(16,), device=device)
            img_fake = model_gen(fixed_noise).detach().cpu()

        plt.imshow(to_pil_image(0.5*img_fake[0] + 0.5), cmap = 'gray')
        plt.axis('off')
        plt.show()
        if(comfort_sentence != "행복"):
            print(f"\n이 글귀를 읽고 마음을 다스려보는건 어떨까요? \n=>{comfort_sentence}\n")
        else:
            print("\n행복한 감정은 우리가 가진 잠재력을 충분히 발휘할 수 있게 도와주면서 우리의 삶을 더욱 풍요롭게 해줘요\n")
            print("앞으로 세상을 살아가면서 슬프고,불안하고,화가나는 일들이 생기면, 차분히 눈을 감고 행복했던 일들을 떠올려보세요!\n")

        print(f"\n{search_query} 플레이리스트를 추천해드릴게요\n")
        for i, result in enumerate(video_results, start=1):
            print(f"{i}. {result['title']} \n {result['url']} \n")

comfort_sentence=""